In [1]:
macro public(expr::Expr)
    @show expr
    return esc(expr)
end

@public (macro with 1 method)

In [3]:
function isafield(expr::Expr)
    return (expr.head == :(::))
end
function parseOOStruct(expr::Expr)
    @assert expr.head == :block
    n = length(expr.args)
    fields = []
    methods = Expr[]
    for i in 1:2:n
        if isafield(expr.args[i+1])
            push!(fields, expr.args[i])
            push!(fields, expr.args[i + 1])
        else
            #push!(methods, expr.args[i])
            push!(methods, expr.args[i + 1])
        end
    end
    fieldsBlock = Expr(:block, fields...)
    return fieldsBlock, methods
end

parseOOStruct (generic function with 1 method)

In [4]:
function genMethodsBlock(methods::Array{Expr, 1}, prevblock::Expr = Expr(:block, ))
    if length(methods) == 0
        return prevblock
    end
    part1 = Expr(:block, prevblock, methods[1])
    if length(methods) == 1
        return part1
    end
    return genMethodsBlock(methods[2:end], part1)
end

genMethodsBlock (generic function with 2 methods)

In [17]:
nfuncdefined = 0
function mygensym()
    global nfuncdefined += 1
    return gensym("OOFunc")
end
    

mygensym (generic function with 1 method)

In [18]:
function genhead(expr::Expr, typeName::Union{Expr, Symbol})
    #@show expr
    sym = mygensym()
    #@show sym
    arg1 = Expr(:(::), :this, typeName)
    newExpr = length(expr.args) == 1 ? Expr(:call, sym, arg1) : Expr(:call, sym, arg1, expr.args[2:end]...)
    if typeof(typeName) == Expr && typeName.head == :curly && length(typeName.args) > 1
        inthatcurl = typeName.args[2:end]
        newExpr = Expr(:where, newExpr, inthatcurl...)
        #@show newExpr
    end
    return newExpr, sym
end

genhead (generic function with 1 method)

In [19]:
function AddPublicFuncToEnv!(methodBlock::Array{Expr, 1}, env::Dict{Symbol, Symbol}, 
        typeName::Union{Expr, Symbol})
    newmethodBlock = Expr[]
    for expr in methodBlock
        if expr.head == :macrocall
            if expr.args[1] == Symbol("@public")
                newHead, sym = genhead(expr.args[3].args[1], typeName)
                newExpr = Expr(:function, newHead, expr.args[3].args[2])
                push!(newmethodBlock, newExpr)
                env[expr.args[3].args[1].args[1]] = sym
            else
                println("expr is not a public func")
                @show expr
            end
        end
    end
    return newmethodBlock
end

AddPublicFuncToEnv! (generic function with 1 method)

In [20]:

env = Dict{Symbol, Symbol}()
macro OO(expr::Expr)
    #@show expr
    typeName = expr.args[2]
    fieldsBlock, methodDefined = parseOOStruct(expr.args[3])
    newmethodBlock = AddPublicFuncToEnv!(methodDefined, env, typeName)
    methodBlock = genMethodsBlock(newmethodBlock)
    #@show methodBlock
    structExpr = Expr(:struct, expr.args[1], expr.args[2], fieldsBlock)
    #@show structExpr
    returnedBlock = Expr(:block, structExpr, methodBlock)
    #@show returnedBlock
    return esc(returnedBlock)
end

macro call(expr::Expr)
    #@show expr
    @assert expr.head == :call
    initialcall = expr.args[1]
    @assert initialcall.head == :. "macro call received $expr"
    objectName, methodName = expr.args[1].args
    methodName = methodName.value
    if length(expr.args) == 1
        #@show Expr(:call, env[methodName], :this)
        return esc(Expr(:call, env[methodName], objectName))
    else
        #@show Expr(:call, env[methodName], :this, expr.args[2:end]...)
        return esc(Expr(:call, env[methodName], objectName, expr.args[2:end]...))
    end
end

@call (macro with 1 method)

In [21]:
@OO mutable struct Node{T}
    value::T
    next::Union{Nothing, Node{T}}
    function Node(value::T) where T
        return new{T}(value, nothing)
    end
    @public function getvalue()
        #println("-"^64)
        return this.value
    end
    @public function getnext()
        #println("*"^64)
        return this.next
    end
end

##OOFunc#294 (generic function with 1 method)

In [22]:
no = Node{Int}(3, nothing)

Node{Int64}(3, nothing)

In [23]:
@call no.getvalue()
no.next = Node{Int}(4, nothing)

Node{Int64}(4, nothing)